<a href="https://colab.research.google.com/github/nfaggian/record_linkage/blob/master/link.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Install dependencies**

In [0]:
!pip install jellyfish tqdm

    100% |████████████████████████████████| 133kB 6.6MB/s 
  Running setup.py bdist_wheel for jellyfish ... - \ | / - \ | / - done
  Stored in directory: /root/.cache/pip/wheels/9c/6f/33/92bb9a4b4562a60ba6a80cedbab8907e48bc7a8b1f369ea0ae
Successfully built jellyfish


In [0]:
!pip install --upgrade -q gspread

In [0]:
from google.colab import auth
auth.authenticate_user()

GCP authentication

In [0]:
from google.colab import auth
auth.authenticate_user()

In [0]:
project_id = 'anz-pso-nfaggian'
sample_size = 500

In [0]:
%env GOOGLE_CLOUD_PROJECT=anz-pso-nfaggian

env: GOOGLE_CLOUD_PROJECT=anz-pso-nfaggian


In [0]:
import pandas as pd

query = f"""
  SELECT
    *
  FROM
     dedup.donors
  WHERE MOD(ABS(FARM_FINGERPRINT(CAST(donor_id AS STRING))), 100) = 0
"""


donors = pd.io.gbq.read_gbq(query, project_id=project_id, dialect='standard', verbose=False)
donors = donors.sort_values(by='donor_id')
donors.head()

,donor_id,last_name,first_name,address_1,address_2,city,state,zip,employer,occupation
2214,60,10-15 cash deposit,None,None,None,None,il,None,None,None
5451,71,101 club,None,108 sir lawrence,None,belleville,il,62221,None,None
5455,74,101 club,None,108 sir lawrence drive,None,belleville,il,62223,None,None
2983,120,1039 s racine,None,1039 s racine,None,chicago,il,60607,None,None
2215,284,11-7-02 cash deposit,None,None,None,None,il,None,None,None


In [0]:
from jellyfish import jaro_distance
import tqdm

In [0]:
data = []

def comparator(record_a, record_b):
  return {
    'donor_id1': record_a.donor_id,
    'donor_id2': record_b.donor_id,
    'first_name_jaro': jaro_distance(record_a.first_name or '', 
                                     record_b.first_name or ''),
    'last_name_jaro': jaro_distance(record_a.last_name or '', 
                                    record_b.last_name or '')
  }


for row_a, record_a in tqdm.tqdm(donors.iterrows()):
  for row_b, record_b in donors.iterrows():
    if row_a == row_b: continue
    data.append(comparator(record_a, record_b)) 
    
    
comparison = pd.DataFrame(data)

1000it [01:52,  8.81it/s]


In [0]:
import altair as alt

alt.Chart(comparison).mark_rect().encode(
    x='donor_id1:O',
    y='donor_id2:O',
    color='last_name_jaro:Q'
)

MaxRowsError: ignored

Chart({
  data:         donor_id1  donor_id2  first_name_jaro  last_name_jaro
  0             847       1835         0.000000        0.541667
  1             847       2866         0.000000        0.497685
  2             847       5408         0.000000        0.489989
  3             847       5409         0.000000        0.489989
  4             847       5413         0.000000        0.481151
  5             847       7366         0.000000        0.291667
  6             847       9725         0.000000        0.429643
  7             847      10379         0.000000        0.500595
  8             847      10380         0.000000        0.495040
  9             847      10974         0.000000        0.454545
  10            847      11017         0.000000        0.481481
  11            847      11028         0.000000        0.500595
  12            847      11033         0.000000        0.466667
  13            847      11046         0.000000        0.466667
  14            847      1